In [1]:
from pyspark.sql import SQLContext
from pyspark.sql import DataFrameNaFunctions
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer

In [2]:
sqlContext = SQLContext(sc)
df = sqlContext.read.load('file:///home/cloudera/Downloads/big-data-4/daily_weather.csv', 
                          format='com.databricks.spark.csv', 
                          header='true',inferSchema='true')
df.columns

['number',
 'air_pressure_9am',
 'air_temp_9am',
 'avg_wind_direction_9am',
 'avg_wind_speed_9am',
 'max_wind_direction_9am',
 'max_wind_speed_9am',
 'rain_accumulation_9am',
 'rain_duration_9am',
 'relative_humidity_9am',
 'relative_humidity_3pm']

In [3]:
featureColumns = ['air_pressure_9am','air_temp_9am','avg_wind_direction_9am','avg_wind_speed_9am',
        'max_wind_direction_9am','max_wind_speed_9am','rain_accumulation_9am',
        'rain_duration_9am']

In [ ]:
print(df.count())
df2 = df.na.drop(subset=['air_temp_9am'])
print(df2.count())

In [ ]:
df = df.drop('number')
df = df.na.drop()
df.count(), len(df.columns)

In [ ]:
binarizer = Binarizer(threshold = 24.99999, inputCol = 'relative_humidity_3pm',
                     outputCol='label')
binarizedDF = binarized.transform(df)

In [ ]:
binarizedDF.select('relative_humidity_3pm','label').show(4)

In [ ]:
assembler = VectorAssembler(inputCols=featureColumns,output='features')
assembed = assembler.transform(binarizedDF)

In [ ]:
(trainingData, testData) = assembled.randomSplit([0.7,0.3], seed= 13234)

In [ ]:
trainingData.count(), testData.count()

In [ ]:
dt = DecisionTreeClassifier(labelCol='label', featuresCol='features',
        maxDepth=5, minInstancesPerNode=20, impurity='gini')

In [ ]:
pipeline = Pipeline(stages=[dt])
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

In [ ]:
predictions.select("prediction","label").show(20)

In [ ]:
predictions.select('prediction','label').write.save(path='file:///home/cloudera/Downloads/big-data-4/predictions.csv',
                        format='com.databricks.spark.csv',
                        header='true')